<a href="https://colab.research.google.com/github/sophiascarano/hybrid_quantum_classical_KiUNet/blob/main/messy_drafts/tf_quantum_class_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass

# import tensorflow as tf
# import numpy as np

# from tensorflow.keras.layers import Layer, InputSpec
# from keras.utils import conv_utils
# import keras.backend as K

clean up these imports...

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
# inherit from this base class
from tensorflow.keras.layers import Layer

############
import functools
from keras import activations
from keras import backend
from keras import constraints
from keras import initializers
from keras import regularizers
from tensorflow.keras.layers import Layer, InputSpec
# imports for backwards namespace compatibility
# pylint: disable=unused-import
from keras.layers.pooling import AveragePooling1D
from keras.layers.pooling import AveragePooling2D
from keras.layers.pooling import AveragePooling3D
from keras.layers.pooling import MaxPooling1D
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import MaxPooling3D
# pylint: enable=unused-import
from keras.utils import conv_utils
#from keras.utils import tf_utils
from tensorflow.python.util.tf_export import keras_export

In [ ]:
import tensorflow.compat.v2 as tf

import functools
from keras import activations
from keras import backend
from keras import constraints
from keras import initializers
from keras import regularizers
from tensorflow.keras.layers import Layer, InputSpec
# imports for backwards namespace compatibility
# pylint: disable=unused-import
from keras.layers.pooling import AveragePooling1D
from keras.layers.pooling import AveragePooling2D
from keras.layers.pooling import AveragePooling3D
from keras.layers.pooling import MaxPooling1D
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import MaxPooling3D
# pylint: enable=unused-import
from keras.utils import conv_utils
#from keras.utils import tf_utils
from tensorflow.python.util.tf_export import keras_export

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import numbers
import os

import numpy as np

from tensorflow.python.eager import context
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import errors_impl
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import ops
from tensorflow.python.framework import random_seed
from tensorflow.python.framework import tensor_shape
from tensorflow.python.framework import tensor_util
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import check_ops
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import random_ops
from tensorflow.python.ops import variables as variables_lib
# go/tf-wildcard-import
# pylint: disable=wildcard-import
from tensorflow.python.ops.gen_nn_ops import *
# pylint: enable=wildcard-import
from tensorflow.python.platform import device_context
from tensorflow.python.util import deprecation
from tensorflow.python.util import dispatch
from tensorflow.python.util.compat import collections_abc
from tensorflow.python.util.deprecation import deprecated_args
from tensorflow.python.util.deprecation import deprecated_argument_lookup

from tensorflow.python.util.tf_export import tf_export

end notes:
- in the documentation, it seems that these helper functions go in a loop.
- also the eager mode / numpy compat makes this unnecessary anyway. Will have to attempt in pytorch next.

In [ ]:
class QConv2D(Layer):
  """
  Args:
    filters: Integer, the dimensionality of the output space (i.e. the number
      of filters in the convolution).
    kernel_size: An integer or tuple/list of 2 integers, specifying the
      dimensions of the convolution window. (width, height)
    groups: A positive integer specifying the number of groups in which the
      input is split along the channel axis. Each group is convolved
      separately with `filters / groups` filters. The output is the
      concatenation of all the `groups` results along the channel axis.
      Input channels and `filters` must both be divisible by `groups`. So depth.
    strides: An integer or tuple/list of n integers, specifying the stride 
      length of the convolution.
    padding: One of `"valid"`, or  `"same"` (case-insensitive).
      `"valid"` means no padding. `"same"` results in padding with zeros
      evenly to the left/right or up/down of the input such that output has the
      same height/width dimension as the input.
    data_format:  data_format: A string, one of `channels_last` (default) or `channels_first`.
      The ordering of the dimensions in the inputs.
      `channels_last` corresponds to inputs with shape
      `(batch_size, ..., channels)` while `channels_first` corresponds to
      inputs with shape `(batch_size, channels, ...)`.
    
  """
  def __init__(self,
               filters,
               kernel_size,
               groups=1,
               strides=1,
               padding='same',
               rank=2,
               data_format='channels_last',
               **kwargs
               ):
    super(QConv2D, self).__init__(**kwargs)
    self.rank = rank

    if isinstance(filters, float):
      filters = int(filters)
    self.filters = filters
    self.groups = groups or 1
    self.kernel_size = conv_utils.normalize_tuple(kernel_size, 
                                                  rank, 
                                                  'kernel_size')
    self.strides = conv_utils.normalize_tuple(strides, rank, 'strides')
    self.padding = conv_utils.normalize_padding(padding)
    #self.data_format = conv_utils.normalize_data_format(data_format) #discontinued
    self.data_format = self._normalize_data_format(data_format)
    #self.data_format = K.normalize_data_format(data_format)
    self.input_spec = InputSpec(min_ndim=self.rank+2) #not sure why? but whatever

    self._validate_init()
    self._channels_first = self.data_format == 'channels_first'
    self._tf_data_format = self._convert_data_format(
        self.data_format, self.rank + 2)

  def _validate_init(self):
    if self.filters is not None and self.filters % self.groups != 0:
      raise ValueError(
          'The number of filters must be evenly divisible by the number of '
          'groups. Received: groups={}, filters={}'.format(
              self.groups, self.filters))

    if not all(self.kernel_size):
      raise ValueError('The argument `kernel_size` cannot contain 0(s). '
                      'Received: %s' % (self.kernel_size,))

    if not all(self.strides):
      raise ValueError('The argument `strides` cannot contains 0(s). '
                      'Received: %s' % (self.strides,))
      
  def build(self, input_shape):
    input_shape = tf.TensorShape(input_shape)
    # input_shape = (1, input_shape[0], input_shape[1], input_shape[2])
    # input_shape = tf.shape(input_shape)
    input_channel = self._get_input_channel(input_shape)
    if input_channel % self.groups !=0:
      raise ValueError(
          'The number of input channels must be evenly divisible by the number '
          'of groups. Received groups={}, but the input has {} channels '
          '(full input shape is {}).'.format(self.groups, input_channel,
                                             input_shape))
    kernel_shape = self.kernel_size + (input_channel // self.groups, 
                                       self.filters)
    
    #their version of adding variables
    # self.kernel = self.add_weight( #from Layer class
    #     name='kernel',
    #     shape=kernel_shape,
    #     initializer=self.kernel_initializer,
    #     regularizer=self.kernel_regularizer,
    #     constraint=self.kernel_constraint,
    #     trainable=True,
    #     dtype=self.dtype)
    
    #using tf.Variable
    # initialize the weights
    w_init = tf.random_normal_initializer()
    self.kernel = tf.Variable(name="kernel",
        initial_value=w_init(shape=(kernel_shape), #not sure if shape is correct
                              dtype='float32'),
        trainable=True)
  
    #back to the example
    channel_axis = self._get_channel_axis()
    self.input_spec = InputSpec(min_ndim=self.rank + 2,
                                axes={channel_axis: input_channel})
    
    # Convert Keras formats to TF native formats
    if isinstance(self.padding, str):
      tf_padding = self.padding.upper()
    else:
      tf_padding = self.padding
    tf_strides = list(self.strides)
    #ignored tf_dilations
    
    ''' I MAY NOT WANT TO GO THROUGH TF CONVOLUTION
    ### lines 1120-1129 of https://github.com/tensorflow/tensorflow/blob/a4dfb8d1a71385bd6d122e4f27f86dcebb96712d/tensorflow/python/ops/nn_ops.py#L1028
    ###   explain this and why you only have to do it for conv1d
    tf_op_name = self.__class__.__name__
    if tf_op_name == 'Conv1D':
      tf_op_name = 'conv1d'  # Backwards compat.

    self._convolution_op = functools.partial(
        tf.nn.convolution,
        strides=tf_strides,
        padding=tf_padding,
        dilations=tf_dilations,
        data_format=self._tf_data_format,
        name=tf_op_name)
    self.built = True
    '''
    #tf_op_name = self.__class__.__name__
    tf_op_name = 'Conv2D'

    self._convolution_op = functools.partial(
        my_quantum_conv,
        strides=tf_strides,
        padding=tf_padding,
        #dilations=tf_dilations,
        data_format=self._tf_data_format,
        name=tf_op_name)
    self.built = True
  
  def call(self, inputs):
    input_shape = inputs.shape
    outputs = self._convolution_op(inputs, self.kernel)
    return outputs

  #other helpful funcs

  def _normalize_data_format(self, value):
    '''
    Instead of conv_utils.normalize_data_format(data_format) above
    Found solution on https://github.com/RaphaelMeudec/deblur-gan/issues/35
      from user sailfish009
    '''
    if value is None:
      value = K.image_data_format()
    data_format = value.lower()
    if data_format not in {'channels_first', 'channels_last'}:
      raise ValueError('The `data_format` argument must be one of '
                         '"channels_first", "channels_last". Received: ' +
                         str(value))
    return data_format

  def _convert_data_format(self, data_format, ndim):
    #line 25 https://github.com/keras-team/keras/blob/master/keras/utils/conv_utils.py
    if data_format == 'channels_last':
      if ndim == 3:
        return 'NWC'
      elif ndim == 4:
        return 'NHWC'
      elif ndim == 5:
        return 'NDHWC'
      else:
        raise ValueError('Input rank not supported:', ndim)
    elif data_format == 'channels_first':
      if ndim == 3:
        return 'NCW'
      elif ndim == 4:
        return 'NCHW'
      elif ndim == 5:
        return 'NCDHW'
      else:
        raise ValueError('Input rank not supported:', ndim)
    else:
      raise ValueError('Invalid data_format:', data_format)

  def _get_channel_axis(self):
    if self.data_format == 'channels_first':
      return -1 - self.rank
    else:
      return -1
  
  def _get_input_channel(self, input_shape):
    channel_axis = self._get_channel_axis()
    if input_shape.dims[channel_axis].value is None:
      raise ValueError('The channel dimension of the inputs '
                       'should be defined. Found `None`.')
    return int(input_shape[channel_axis])


In [ ]:
def matrix_mult(image_matrix, filter_matrix):
  return image_matrix * filter_matrix

In [ ]:
def convolution_op(one_input, one_filter):
  '''takes one image and convolves one filter around that image, returning feature map'''


In [ ]:
def my_quantum_conv(inputs, 
                    filters, 
                    strides=1, 
                    padding='VALID', 
                    data_format='NHWC',
                    dilations=None, 
                    name='QConv2D'):
  return inputs
  # return tf.nn.convolution(input, 
  #                          filters, 
  #                          strides=strides, 
  #                          padding=padding, 
  #                          data_format=data_format,
  #                          dilations=dilations, 
  #                          name='Conv2D')


In [ ]:
# identity block within residual
class IdentityBlock(tf.keras.Model):
  def __init__(self, filters, kernel_size):
    super(IdentityBlock, self).__init__(name='')


    self.conv1 = QConv2D(filters, kernel_size, groups=1, strides=1, padding='same', rank=2, data_format=None)
    self.bn1 = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.bn1(x)

    return x

class ResNetMini(tf.keras.Model):
  def __init__(self, num_classes):
    super(ResNetMini, self).__init__()
    self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.activation = tf.keras.layers.Activation('relu')
    self.max_pool = tf.keras.layers.MaxPool2D((3,3))
    self.identity1a = IdentityBlock(64, 3)
    self.identity1b = IdentityBlock(64, 3)
    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
    self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    # x = self.conv(inputs)
    # x = self.bn(x)
    # x = self.activation(x)
    # x = self.max_pool(x)

    # x = self.identity1a(x)
    # x = self.identity1b(x)

    x = self.identity1a(inputs)

    x = self.global_pool(x)
    x = self.classifier(x)

    return x

# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# now actually run it I guess?

resnet_mini = ResNetMini(10)
resnet_mini.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
dataset = tfds.load('mnist', split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

In [ ]:
resnet_mini.fit(dataset, epochs=1)

1875/1875 [==============================] - 14s 7ms/step - loss: 2.2686 - accuracy: 0.1353


In [ ]:
# maybe this is the correct conv2d?
from tensorflow.python.keras import backend_config
image_data_format = backend_config.image_data_format
def conv2d(x,
           kernel,
           strides=(1, 1),
           padding='valid',
           data_format=None,
           dilation_rate=(1, 1)):
  
  if data_format is None:
    data_format = image_data_format() #imported above
  if data_format not in {'channels_first', 'channels_last'}:
    raise ValueError('Unknown data_format: ' + str(data_format))

  x, tf_data_format = _preprocess_conv2d_input(x, data_format) #defined below
  padding = _preprocess_padding(padding) #defined below
  x = nn.convolution( #WHAT ARE YOU KIDDING
      input=x,
      filter=kernel,
      dilation_rate=dilation_rate,
      strides=strides,
      padding=padding,
      data_format=tf_data_format)
  if data_format == 'channels_first' and tf_data_format == 'NHWC':
    x = array_ops.transpose(x, (0, 3, 1, 2))  # NHWC -> NCHW
  return x

def _preprocess_conv2d_input(x, data_format, force_transpose=False):
  tf_data_format = 'NHWC'
  if data_format == 'channels_first':
    if not _has_nchw_support() or force_transpose:
      x = array_ops.transpose(x, (0, 2, 3, 1))  # NCHW -> NHWC
    else:
      tf_data_format = 'NCHW'
  return x, tf_data_format

def _preprocess_padding(padding):
  if padding == 'same':
    padding = 'SAME'
  elif padding == 'valid':
    padding = 'VALID'
  else:
    raise ValueError('Invalid padding: ' + str(padding))
  return padding

In [ ]:
_CHANNELS_LAST_FORMATS = frozenset({
    "NWC", "NHC", "NHWC", "NWHC", "NDHWC", "NDWHC", "NHDWC", "NHWDC", "NWDHC",
    "NWHDC"
})

def convolution(
    input,  # pylint: disable=redefined-builtin
    filter,  # pylint: disable=redefined-builtin
    padding,
    strides=None,
    dilation_rate=None,
    name=None,
    data_format=None,
    filters=None,
    dilations=None):  # pylint: disable=g-doc-args

  filter = deprecated_argument_lookup("filters", filters, "filter", filter)
  dilation_rate = deprecated_argument_lookup(
      "dilations", dilations, "dilation_rate", dilation_rate)
  return convolution_internal(
      input,
      filter,
      strides=strides,
      padding=padding,
      data_format=data_format,
      dilations=dilation_rate,
      name=name)

def convolution_internal(
    input,  # pylint: disable=redefined-builtin
    filters,
    strides=None,
    padding="VALID",
    data_format=None,
    dilations=None,
    name=None,
    call_from_convolution=True,
    num_spatial_dims=None):
  
  filters = ops.convert_to_tensor(filters, name='filters')
  input = ops.convert_to_tensor(input, name="input")

  filters_rank = filters.shape.rank
  inputs_rank = input.shape.rank

  if num_spatial_dims is None:
    if filters_rank:
      num_spatial_dims = filters_rank - 2
    elif inputs_rank:
      num_spatial_dims = inputs_rank - 2
    else:
      raise ValueError("rank of input or filter must be known")
  elif filters_rank and filters_rank - 2 != num_spatial_dims:
    raise ValueError(
        "inconsistent estimate of spatial dims ({}) vs. actual passed "
        "num_spatial_dims ({}).  n was estimated as len(filters.shape) - 2, "
        "but filters shape is: {}".format(filters_rank, num_spatial_dims,
                                          filters.shape))

  if inputs_rank:
    num_batch_dims = inputs_rank - num_spatial_dims - 1  # Channel dimension.
  else:
    num_batch_dims = 1  # By default, assume single batch dimension.

  if num_spatial_dims not in {1, 2, 3}:
    raise ValueError(
        "num_spatial_dims (input.shape.ndims - num_batch_dims - 1) must be one "
        "of 1, 2 or 3 but saw {}.  num_batch_dims: {}.".format(
            num_spatial_dims, num_batch_dims))
  
  if data_format is None or data_format in _CHANNELS_LAST_FORMATS: #defined above
    channel_index = num_batch_dims + num_spatial_dims
  else:
    channel_index = num_batch_dims

  ############using _get_sequence for dilations stuff
  if dilations is None:
    dilations = _get_sequence(dilations, num_spatial_dims, channel_index,
                              "dilations")
    is_dilated_conv = False
  else:
    dilations = _get_sequence(dilations, num_spatial_dims, channel_index,
                              "dilations")
    is_dilated_conv = any(i != 1 for i in dilations)
  ##########
  strides = _get_sequence(strides, num_spatial_dims, channel_index, "strides")
  '''
  has_tpu_context = device_context.enclosing_tpu_context() is not None
  '''
  if name:
    default_name = None
  '''
  elif not has_tpu_context or call_from_convolution:
    default_name = "convolution" '''
  elif num_spatial_dims == 2:  # Most common case.
    default_name = "Conv2D"
  elif num_spatial_dims == 3:
    default_name = "Conv3D"
  else:
    default_name = "conv1d"

  with ops.name_scope(name, default_name, [input, filters]) as name:
    # Fast path for TPU or if no dilation, as gradient only supported on TPU
    # for dilations.
    if not is_dilated_conv: ''' or has_tpu_context:'''
      if num_spatial_dims == 2:  # Most common case.
        op = _conv2d_expanded_batch #defined below
      '''elif num_spatial_dims == 3:
        op = _conv3d_expanded_batch'''
      else:
        '''op = conv1d'''
        raise ValueError('num_spatial_dims isnt equal to 2')

      return op(
          input,
          filters,
          strides,
          padding=padding,
          data_format=data_format,
          dilations=dilations,
          name=name)
    else:
      '''if channel_index == 1:
        strides = strides[2:]
        dilations = dilations[2:]
      else:
        strides = strides[1:-1]
        dilations = dilations[1:-1]

      op = Convolution(
          tensor_shape.as_shape(input.shape),
          tensor_shape.as_shape(filters.shape),
          padding,
          strides=strides,
          dilation_rate=dilations,
          name=name,
          data_format=data_format,
          num_spatial_dims=num_spatial_dims)
      return op(input, filters)'''
      raise ValueError('check dilations issue in convolution_internal')


def _get_sequence(value, n, channel_index, name):
  """Formats a value input for gen_nn_ops."""
  # Performance is fast-pathed for common cases:
  # `None`, `list`, `tuple` and `int`.
  if value is None:
    return [1] * (n + 2)

  # Always convert `value` to a `list`.
  if isinstance(value, list):
    pass
  elif isinstance(value, tuple):
    value = list(value)
  elif isinstance(value, int):
    value = [value]
  elif not isinstance(value, collections_abc.Sized):
    value = [value]
  else:
    value = list(value)  # Try casting to a list.

  len_value = len(value)

  # Fully specified, including batch and channel dims.
  if len_value == n + 2:
    return value

  # Apply value to spatial dims only.
  if len_value == 1:
    value = value * n  # Broadcast to spatial dimensions.
  elif len_value != n:
    raise ValueError("{} should be of length 1, {} or {} but was {}".format(
        name, n, n + 2, len_value))

  # Add batch and channel dims (always 1).
  if channel_index == 1:
    return [1, 1] + value
  else:
    return [1] + value + [1]

def _conv2d_expanded_batch(
    input,  # pylint: disable=redefined-builtin
    filters,
    strides,
    padding,
    data_format,
    dilations,
    name):
  """Helper function for `convolution_internal`; handles expanded batches."""
  # Try really hard to avoid modifying the legacy name scopes - return early.
  '''input_rank = input.shape.rank
  if input_rank is None or input_rank < 5:'''
    # We avoid calling squeeze_batch_dims to reduce extra python function
    # call slowdown in eager mode.  This branch doesn't require reshapes.
  return conv2d(
      input,
      filter=filters,
      strides=strides,
      padding=padding,
      data_format=data_format,
      dilations=dilations,
      name=name)
  '''return squeeze_batch_dims(
      input,
      functools.partial(
          conv2d,
          filter=filters,
          strides=strides,
          padding=padding,
          data_format=data_format,
          dilations=dilations),
      inner_rank=3,
      name=name)'''

def conv2d(inputs,
           filters,
           kernel_size,
           strides=(1, 1),
           padding='valid',
           data_format='channels_last',
           dilation_rate=(1, 1),
           activation=None,
           use_bias=True,
           kernel_initializer=None,
           bias_initializer=init_ops.zeros_initializer(),
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None,
           trainable=True,
           name=None,
           reuse=None):
  warnings.warn('`tf.layers.conv2d` is deprecated and '
                'will be removed in a future version. '
                'Please Use `tf.keras.layers.Conv2D` instead.')
  layer = Conv2D(
      filters=filters,
      kernel_size=kernel_size,
      strides=strides,
      padding=padding,
      data_format=data_format,
      dilation_rate=dilation_rate,
      activation=activation,
      use_bias=use_bias,
      kernel_initializer=kernel_initializer,
      bias_initializer=bias_initializer,
      kernel_regularizer=kernel_regularizer,
      bias_regularizer=bias_regularizer,
      activity_regularizer=activity_regularizer,
      kernel_constraint=kernel_constraint,
      bias_constraint=bias_constraint,
      trainable=trainable,
      name=name,
      _reuse=reuse,
      _scope=name)
  return layer.apply(inputs)

In [ ]:

# utility function to normalize the images and return (image, label) pairs.
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

# now actually run it I guess?

resnet_mini = ResNetMini(10)
resnet_mini.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
dataset = tfds.load('mnist', split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)
resnet_mini.fit(dataset, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3342 - accuracy: 0.9076
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0837 - accuracy: 0.9756


In [ ]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 1

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
resnet_mini = ResNetMini(10)
resnet_mini.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
#dataset = tfds.load('mnist', split=tfds.Split.TRAIN)
#dataset = dataset.map(preprocess).batch(32)
resnet_mini.fit(x_train, y_train, epochs=1)

ValueError: ignored

In [ ]:
import tensorflow.compat.v2 as tf

import functools
from keras import activations
from keras import backend
from keras import constraints
from keras import initializers
from keras import regularizers
from keras.engine.base_layer import Layer
from keras.engine.input_spec import InputSpec
# imports for backwards namespace compatibility
# pylint: disable=unused-import
from keras.layers.pooling import AveragePooling1D
from keras.layers.pooling import AveragePooling2D
from keras.layers.pooling import AveragePooling3D
from keras.layers.pooling import MaxPooling1D
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import MaxPooling3D
# pylint: enable=unused-import
from keras.utils import conv_utils
from keras.utils import tf_utils
from tensorflow.python.util.tf_export import keras_export

In [ ]:
@keras_export('keras.layers.Conv2D', 'keras.layers.Convolution2D')
class Conv2D(Conv):
  """2D convolution layer (e.g. spatial convolution over images).
  This layer creates a convolution kernel that is convolved
  with the layer input to produce a tensor of
  outputs. If `use_bias` is True,
  a bias vector is created and added to the outputs. Finally, if
  `activation` is not `None`, it is applied to the outputs as well.
  When using this layer as the first layer in a model,
  provide the keyword argument `input_shape`
  (tuple of integers or `None`, does not include the sample axis),
  e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
  in `data_format="channels_last"`. You can use `None` when
  a dimension has variable size.
  Examples:
  >>> # The inputs are 28x28 RGB images with `channels_last` and the batch
  >>> # size is 4.
  >>> input_shape = (4, 28, 28, 3)
  >>> x = tf.random.normal(input_shape)
  >>> y = tf.keras.layers.Conv2D(
  ... 2, 3, activation='relu', input_shape=input_shape[1:])(x)
  >>> print(y.shape)
  (4, 26, 26, 2)
  >>> # With `dilation_rate` as 2.
  >>> input_shape = (4, 28, 28, 3)
  >>> x = tf.random.normal(input_shape)
  >>> y = tf.keras.layers.Conv2D(
  ... 2, 3, activation='relu', dilation_rate=2, input_shape=input_shape[1:])(x)
  >>> print(y.shape)
  (4, 24, 24, 2)
  >>> # With `padding` as "same".
  >>> input_shape = (4, 28, 28, 3)
  >>> x = tf.random.normal(input_shape)
  >>> y = tf.keras.layers.Conv2D(
  ... 2, 3, activation='relu', padding="same", input_shape=input_shape[1:])(x)
  >>> print(y.shape)
  (4, 28, 28, 2)
  >>> # With extended batch shape [4, 7]:
  >>> input_shape = (4, 7, 28, 28, 3)
  >>> x = tf.random.normal(input_shape)
  >>> y = tf.keras.layers.Conv2D(
  ... 2, 3, activation='relu', input_shape=input_shape[2:])(x)
  >>> print(y.shape)
  (4, 7, 26, 26, 2)
  Args:
    filters: Integer, the dimensionality of the output space (i.e. the number of
      output filters in the convolution).
    kernel_size: An integer or tuple/list of 2 integers, specifying the height
      and width of the 2D convolution window. Can be a single integer to specify
      the same value for all spatial dimensions.
    strides: An integer or tuple/list of 2 integers, specifying the strides of
      the convolution along the height and width. Can be a single integer to
      specify the same value for all spatial dimensions. Specifying any stride
      value != 1 is incompatible with specifying any `dilation_rate` value != 1.
    padding: one of `"valid"` or `"same"` (case-insensitive).
      `"valid"` means no padding. `"same"` results in padding with zeros evenly
      to the left/right or up/down of the input such that output has the same
      height/width dimension as the input.
    data_format: A string, one of `channels_last` (default) or `channels_first`.
      The ordering of the dimensions in the inputs. `channels_last` corresponds
      to inputs with shape `(batch_size, height, width, channels)` while
      `channels_first` corresponds to inputs with shape `(batch_size, channels,
      height, width)`. It defaults to the `image_data_format` value found in
      your Keras config file at `~/.keras/keras.json`. If you never set it, then
      it will be `channels_last`.
    dilation_rate: an integer or tuple/list of 2 integers, specifying the
      dilation rate to use for dilated convolution. Can be a single integer to
      specify the same value for all spatial dimensions. Currently, specifying
      any `dilation_rate` value != 1 is incompatible with specifying any stride
      value != 1.
    groups: A positive integer specifying the number of groups in which the
      input is split along the channel axis. Each group is convolved separately
      with `filters / groups` filters. The output is the concatenation of all
      the `groups` results along the channel axis. Input channels and `filters`
      must both be divisible by `groups`.
    activation: Activation function to use. If you don't specify anything, no
      activation is applied (see `keras.activations`).
    use_bias: Boolean, whether the layer uses a bias vector.
    kernel_initializer: Initializer for the `kernel` weights matrix (see
      `keras.initializers`). Defaults to 'glorot_uniform'.
    bias_initializer: Initializer for the bias vector (see
      `keras.initializers`). Defaults to 'zeros'.
    kernel_regularizer: Regularizer function applied to the `kernel` weights
      matrix (see `keras.regularizers`). 
    bias_regularizer: Regularizer function applied to the bias vector (see
      `keras.regularizers`). 
    activity_regularizer: Regularizer function applied to the output of the
      layer (its "activation") (see `keras.regularizers`).
    kernel_constraint: Constraint function applied to the kernel matrix (see
      `keras.constraints`).
    bias_constraint: Constraint function applied to the bias vector (see
      `keras.constraints`).
  Input shape:
    4+D tensor with shape: `batch_shape + (channels, rows, cols)` if
      `data_format='channels_first'`
    or 4+D tensor with shape: `batch_shape + (rows, cols, channels)` if
      `data_format='channels_last'`.
  Output shape:
    4+D tensor with shape: `batch_shape + (filters, new_rows, new_cols)` if
    `data_format='channels_first'` or 4+D tensor with shape: `batch_shape +
      (new_rows, new_cols, filters)` if `data_format='channels_last'`.  `rows`
      and `cols` values might have changed due to padding.
  Returns:
    A tensor of rank 4+ representing
    `activation(conv2d(inputs, kernel) + bias)`.
  Raises:
    ValueError: if `padding` is `"causal"`.
    ValueError: when both `strides > 1` and `dilation_rate > 1`.
  """

  def __init__(self,
               filters,
               kernel_size,
               strides=(1, 1),
               padding='valid',
               data_format=None,
               dilation_rate=(1, 1),
               groups=1,
               activation=None,
               use_bias=True,
               kernel_initializer='glorot_uniform',
               bias_initializer='zeros',
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               **kwargs):
    super(Conv2D, self).__init__(
        rank=2,
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        data_format=data_format,
        dilation_rate=dilation_rate,
        groups=groups,
        activation=activations.get(activation),
        use_bias=use_bias,
        kernel_initializer=initializers.get(kernel_initializer),
        bias_initializer=initializers.get(bias_initializer),
        kernel_regularizer=regularizers.get(kernel_regularizer),
        bias_regularizer=regularizers.get(bias_regularizer),
        activity_regularizer=regularizers.get(activity_regularizer),
        kernel_constraint=constraints.get(kernel_constraint),
        bias_constraint=constraints.get(bias_constraint),
        **kwargs)